In [1]:
import numpy as np 
import pandas as pd 
import json
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import glob
from skimage.feature import hog
from PIL import Image
import os.path, sys
from skimage.io import imread, imshow
import cv2
from skimage import data, color, feature , exposure
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns; sns.set()

Using TensorFlow backend.


In [2]:
test_data = pd.read_csv('/Users/toothless/Downloads/Thesis/data/test_labels/test_data_labels.csv')

filename = test_data['new_names'].values

path = '/Users/toothless/Downloads/Thesis/data/Train_Test/testoriginal/'

In [3]:
test_images = []
for file in filename:
    test_images.append(path+file)

In [4]:
test_re = []
final_size=150
for img in test_images:
    im = Image.open(img)
    f, e = os.path.splitext(img)
    size = im.size
    ratio = float(final_size) / max(size)
    new_image_size = tuple([int(x*ratio) for x in size])
    im = im.resize(new_image_size, Image.ANTIALIAS)
    new_im = Image.new("RGB", (final_size, final_size))
    new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
    test_re.append(np.asarray(new_im))
  

In [5]:
X_test = np.array(test_re)
Y_test = np.array(test_data['labels'])

In [6]:
X_test.shape

(1867, 150, 150, 3)

In [7]:
Y_test.shape

(1867,)

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D , Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn import metrics
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [9]:
def build_cnn():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150,150,3) , activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(256 ,  activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.2))
    model.add(Dense(2 , activation ='softmax'))

    return model

In [11]:
model = build_cnn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [22]:
model.load_weights('/Users/toothless/Downloads/Thesis/FinalResults/RAW_CNN/model_RAW_v5.hdf5')

In [23]:
y_pred = model.predict_classes(X_test)

print('Best CV Fold Accuracy %.2f%%' % (metrics.accuracy_score(Y_test, y_pred)*100))

Best CV Fold Accuracy 65.99%
